In [1]:
import sqlite3

db = sqlite3.connect('dictionary.db')
c = db.cursor()
tokens = dict()

for t in c.execute('SELECT * FROM tokens'):
    t_id = t[0]
    letter = t[1]
    pos = t[2]
    tokens[(letter, pos)] = t_id

In [2]:
import simplejson as json
parsed_corpus = open('parsed_corpus.txt', encoding='utf-8')

sy_3lgram = dict()
sy_3rgram = dict()
sy_3gram= dict()

In [3]:
def process_sentence(stc):
    stc.insert(0, {'letter': '<시작>', 'pos': [{'letter': '<시작>', 'tag': 'ZST'}]})
    stc.append({'letter': '<끝>', 'pos': [{'letter': '<끝>', 'tag': 'ZED'}]})
    
    for i, l in enumerate(stc):
        letter = l['letter']
        pos = tuple((t['letter'], t['tag']) for t in l['pos'])
        pos = tuple(tokens[p] for p in pos)
        
        if len(letter) > 1:
            continue
            
        left3 = ''.join([l['letter'] for l in stc[i-2:i+1]])
        center3 = ''.join([l['letter'] for l in stc[i-1:i+2]])
        right3 = ''.join([l['letter'] for l in stc[i:i+3]])

        if left3 not in sy_3lgram:
            sy_3lgram[left3] = dict()
        if center3 not in sy_3gram:
            sy_3gram[center3] = dict()
        if right3 not in sy_3rgram:
            sy_3rgram[right3] = dict()
            
        sy_3lgram[left3][pos] = sy_3lgram[left3][pos] + 1 if pos in sy_3lgram[left3] else 1
        sy_3gram[center3][pos] = sy_3gram[center3][pos] + 1 if pos in sy_3gram[center3] else 1
        sy_3rgram[right3][pos] = sy_3rgram[right3][pos] + 1 if pos in sy_3rgram[right3] else 1

In [4]:
line = parsed_corpus.readline()
count = 0
while line != '' and line != '\n':
    process_sentence(json.loads(line))
    del line
    line = parsed_corpus.readline()
    count += 1
    if count % 10000 == 0:
        print(count)
print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000


In [5]:
c.execute('CREATE TABLE `left_3gram` ( `letter` TEXT, `tokens` TEXT, `count` INTEGER )')
c.execute('CREATE INDEX `letter_left` ON `left_3gram` (`letter` ASC)')

c.execute('CREATE TABLE `center_3gram` ( `letter` TEXT, `tokens` TEXT, `count` INTEGER )')
c.execute('CREATE INDEX `letter_center` ON `center_3gram` (`letter` ASC)')

c.execute('CREATE TABLE `right_3gram` ( `letter` TEXT, `tokens` TEXT, `count` INTEGER )')
c.execute('CREATE INDEX `letter_right` ON `right_3gram` (`letter` ASC)')

db.commit()

In [6]:
for gram in sy_3lgram:
    letter = gram
    for tokens, count in sy_3lgram[gram].items():
        tokens_str = ','.join([str(t) for t in tokens])
        c.execute('INSERT INTO `left_3gram` VALUES (?, ?, ?)', (letter, tokens_str, count))
for gram in sy_3gram:
    letter = gram
    for tokens, count in sy_3gram[gram].items():
        tokens_str = ','.join([str(t) for t in tokens])
        c.execute('INSERT INTO `center_3gram` VALUES (?, ?, ?)', (letter, tokens_str, count))
for gram in sy_3rgram:
    letter = gram
    for tokens, count in sy_3rgram[gram].items():
        tokens_str = ','.join([str(t) for t in tokens])
        c.execute('INSERT INTO `right_3gram` VALUES (?, ?, ?)', (letter, tokens_str, count))

db.commit()
db.close()